In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import random

In [2]:
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras.layers import Dropout
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.utils import Sequence

Using TensorFlow backend.


In [3]:
df = pd.read_csv('csv/ext_face_flg.csv')
df.head()

,name,file_path,x0,x1,y0,y1,file_name,file,flg
0,三上悠亜,images_best_titles/三上悠亜/ssni00409jp-9.jpg,450,544,96,190,ssni00409jp-9_0.jpg,face_true/ssni00409jp-9_0.jpg,1
1,三上悠亜,images_best_titles/三上悠亜/ssni00409jp-10.jpg,426,507,130,211,ssni00409jp-10_0.jpg,face_true/ssni00409jp-10_0.jpg,1
2,三上悠亜,images_best_titles/三上悠亜/sivr00016jp-1.jpg,359,472,65,178,sivr00016jp-1_0.jpg,face_true/sivr00016jp-1_0.jpg,1
3,三上悠亜,images_best_titles/三上悠亜/sivr00016jp-4.jpg,245,483,110,348,sivr00016jp-4_0.jpg,face_true/sivr00016jp-4_0.jpg,1
4,三上悠亜,images_best_titles/三上悠亜/sivr00016jp-6.jpg,369,476,54,161,sivr00016jp-6_0.jpg,face_true/sivr00016jp-6_0.jpg,1


In [4]:
1-df['flg'].mean()

0.7746061415220293

In [5]:
size = (224, 224)

In [6]:
data_paths = df['file'].values
data_classes = df['flg'].values

In [7]:
num = len(data_paths)
pickup = np.array(random.sample(range(num),num))
paths_shaffuled = data_paths[pickup]
classes_shaffuled = data_classes[pickup]

In [8]:
class MyGenerator(Sequence):
    """Custom generator"""

    def __init__(self, data_paths, data_classes, 
                 batch_size=16, size=(224,224), ch=3, num_of_class=2):
        """construction   
        :param data_paths: List of image file  
        :param data_classes: List of class  
        :param batch_size: Batch size  
        :param width: Image width  
        :param height: Image height  
        :param ch: Num of image channels  
        :param num_of_class: Num of classes  
        """
        self.data_paths = data_paths
        self.data_classes = data_classes
        self.length = len(data_paths)
        self.batch_size = batch_size
        self.size = size
        self.ch = ch
        self.num_of_class = num_of_class
        self.num_batches_per_epoch = int((self.length - 1) / batch_size) + 1

    def __getitem__(self, idx):
        """Get batch data   

        :param idx: Index of batch  

        :return imgs: numpy array of images 
        :return labels: numpy array of label  
        """
        start_pos = self.batch_size * idx
        end_pos = start_pos + self.batch_size
        if end_pos > self.length:
            end_pos = self.length
        item_paths = self.data_paths[start_pos : end_pos]
        item_classes = self.data_classes[start_pos : end_pos]
        imgs = np.empty((len(item_paths), self.size[0], self.size[1], self.ch), dtype=np.float32)
        labels = np.zeros((len(item_paths), self.num_of_class), dtype=np.float32)

        for i, (item_path, item_class) in enumerate(zip(item_paths, item_classes)):
            img = cv2.imread(item_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.size)
            imgs[i, :] = img
            labels[i, item_class] = 1
        return imgs, labels

    def __len__(self):
        """Batch length"""
        return self.num_batches_per_epoch

    def on_epoch_end(self):
        """Task when end of epoch"""
        pass


In [9]:
inputs = Input(shape=(size[0],size[1],3), name='input')

dense_list = []

## Block 1
conv1_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_1')(inputs)
norm1_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_1')(conv1_1)
conv1_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_2')(norm1_1)
norm1_2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_2')(conv1_2)

pool1 = MaxPooling2D((2, 2),strides=(2, 2),padding='same',name='pool1')(norm1_2)
drop1 = Dropout(0.2)(pool1)

## Block 2
conv2_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_1')(drop1)
norm2_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_1')(conv2_1)
conv2_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_2')(norm2_1)
norm2_2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_2')(conv2_2)
pool2 = MaxPooling2D((2, 4), strides=(3, 3), padding='same',name='pool2')(norm2_2)
drop2 = Dropout(0.2)(pool2)
flatten2 = Flatten(name='flatten2')(drop2)
dense2 = Dense(80, name='dence2')(flatten2)
dense_list.append(dense2)

dense = Dense(2, name='dense_all')(dense2)
pred = Activation('softmax',name='pred')(dense)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')

model = Model(inputs=inputs, outputs=pred)
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
val_count = int(num * 0.2)
train_gen = MyGenerator(
                             paths_shaffuled[val_count:], 
                             classes_shaffuled[val_count:], 
                             batch_size=8)
val_gen = MyGenerator(
                             paths_shaffuled[:val_count], 
                             classes_shaffuled[:val_count], 
                             batch_size=8)

In [11]:
import multiprocessing

# 同時実行プロセス数
process_count = multiprocessing.cpu_count() - 1

In [12]:
model.fit_generator(
           train_gen, 
           steps_per_epoch=train_gen.num_batches_per_epoch, 
           validation_data=val_gen, 
           validation_steps=val_gen.num_batches_per_epoch,
           epochs=3,
           shuffle=True)

Epoch 1/3
3745/3745 [==============================] - 734s 196ms/step - loss: 3.5671 - acc: 0.7774 - val_loss: 3.8013 - val_acc: 0.7628
Epoch 2/3
3745/3745 [==============================] - 725s 194ms/step - loss: 3.5667 - acc: 0.7775 - val_loss: 3.8003 - val_acc: 0.7629
Epoch 3/3
1524/3745 [===========>..................] - ETA: 6:38 - loss: 3.5303 - acc: 0.7798

KeyboardInterrupt: 

In [13]:
model.predict(X_train[-1:])

NameError: name 'X_train' is not defined

In [5]:
size = (112, 112)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False)

In [7]:
X_list = []
y_list = []
for i in range(len(df)):
    file_path = df['file'].iloc[i]
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    X_list.append(img)
    y_list.append(df['flg'].iloc[i])
X = np.array(X_list)
y = np.array(y_list)

In [9]:
import pickle
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('t_train.pickle', 'wb') as f:
    pickle.dump(t_train, f)

In [5]:
import pickle
with open('X_train.pickle', 'rb') as f:
    X_train = pickle.load(f)
with open('t_train.pickle', 'rb') as f:
    t_train = pickle.load(f)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# from sklearn.preprocessing import OneHotEncoder

# enc = OneHotEncoder(handle_unknown='ignore')
# t_train_ = enc.fit_transform(y_train_.reshape([-1,1]))
# t_val = enc.transform(t_val.reshape([-1,1]))

In [8]:
# plt.figure(figsize=(10,5))
# plt.subplot(121)
# plt.imshow(image)
# plt.subplot(122)
# plt.imshow(img)
# plt.show()

In [11]:
inputs = Input(shape=(size[0],size[1],3), name='input')

dense_list = []

## Block 1
conv1_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_1')(inputs)
norm1_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_1')(conv1_1)
conv1_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_2')(norm1_1)
norm1_2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_2')(conv1_2)

pool1 = MaxPooling2D((2, 2),strides=(2, 2),padding='same',name='pool1')(norm1_2)
drop1 = Dropout(0.2)(pool1)
# flatten1 = Flatten(name='flatten1')(drop1)
# dense1 = Dense(80, name='dence1')(flatten1)
# dense_list.append(dense1)
#pred1 = Activation('softmax',name='prediction1')(dense1)

## Block 2
conv2_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_1')(drop1)
norm2_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_1')(conv2_1)
conv2_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_2')(norm2_1)
norm2_2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_2')(conv2_2)
pool2 = MaxPooling2D((2, 4), strides=(3, 3), padding='same',name='pool2')(norm2_2)
drop2 = Dropout(0.2)(pool2)
flatten2 = Flatten(name='flatten2')(drop2)
dense2 = Dense(80, name='dence2')(flatten2)
dense_list.append(dense2)
#pred2 = Activation('softmax',name='prediction2')(dense2)

## Block 3
# conv3_1 = Conv2D(8, (3, 3),activation='relu',padding='same',name='conv3_1')(drop2)
# conv3_2 = Conv2D(8, (3, 3),activation='relu',padding='same',name='conv3_2')(conv3_1)
# pool3 = AveragePooling2D((2, 4), strides=(1, 2), padding='same',name='pool3')(conv3_2)
# drop3 = Dropout(0.2)(pool3)
# flatten3 = Flatten(name='flatten3')(drop3)
# dense3 = Dense(80, name='dence3')(flatten3)
# dense_list.append(dense3)

# ## Block 3
# conv4_1 = Conv2D(2, (3, 3),activation='relu',padding='same',name='conv4_1')(drop3)
# conv4_2 = Conv2D(2, (3, 3),activation='relu',padding='same',name='conv4_2')(conv4_1)
# pool4 = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool4')(conv4_2)
# drop4 = Dropout(0.25)(pool4)
# flatten4 = Flatten(name='flatten4')(drop4)
# dense4 = Dense(80, name='dence4')(flatten4)
# dense_list.append(dense4)

# ## Block 3
# conv5_1 = Conv2D(2, (3, 3),activation='relu',padding='same',name='conv5_1')(pool4)
# conv5_2 = Conv2D(2, (3, 3),activation='relu',padding='same',name='conv5_2')(conv5_1)
# pool5 = MaxPooling2D((2, 2), strides=(1, 2), padding='same',name='pool5')(conv5_2)
# drop5 = Dropout(0.25)(pool5)
# flatten5 = Flatten(name='flatten5')(drop5)
# dense5 = Dense(80, name='dence5')(flatten5)
# dense_list.append(dense5)


# dense_merge = concatenate(dense_list, name='pred_mix', axis=1)
dense = Dense(2, name='dense_all')(dense2)
pred = Activation('softmax',name='pred')(dense)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')

model = Model(inputs=inputs, outputs=pred)
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
#model.summary()

In [9]:
from keras.utils import np_utils
y_train_ = np_utils.to_categorical(y_train, 2)
y_val_ = np_utils.to_categorical(y_val, 2)

In [10]:
batch_size = 16
epochs = 5

datagen.fit(X_train)

In [15]:

model.fit_generator(datagen.flow(X_train, y_train_, batch_size=batch_size),
                    steps_per_epoch=int(len(X_train) /batch_size),
                    validation_data=datagen.flow(X_val, y_val_, batch_size=batch_size),
                    epochs=epochs)

Epoch 1/5
1872/1872 [==============================] - 186s 99ms/step - loss: 3.6107 - acc: 0.7746 - val_loss: 3.6170 - val_acc: 0.7744
Epoch 2/5
1872/1872 [==============================] - 185s 99ms/step - loss: 3.6120 - acc: 0.7747 - val_loss: 3.6170 - val_acc: 0.7744
Epoch 3/5
  23/1872 [..............................] - ETA: 2:49 - loss: 3.7462 - acc: 0.7663

KeyboardInterrupt: 

In [12]:
model.fit(X_train_, t_train_,  batch_size=16, nb_epoch=5, verbose=1, validation_data=(X_val, t_val))

/home/ryoji/Documents/python/rec_act/scrape/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 29960 samples, validate on 7490 samples
Epoch 1/5
29960/29960 [==============================] - 185s 6ms/step - loss: 3.6114 - acc: 0.7737 - val_loss: 3.6170 - val_acc: 0.7744
Epoch 2/5
29960/29960 [==============================] - 185s 6ms/step - loss: 3.6122 - acc: 0.7747 - val_loss: 3.6170 - val_acc: 0.7744
Epoch 3/5
11664/29960 [==========>...................] - ETA: 1:43 - loss: 3.5320 - acc: 0.7797

KeyboardInterrupt: 

In [16]:
model.predict(X_train[-1:])

array([[1., 0.]], dtype=float32)

In [11]:
inputs = Input(shape=(size[0],size[1],3), name='input')
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)


# VGG16の図の緑色の部分（FC層）の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(2, activation='softmax'))

# VGG16とFC層を結合してモデルを作成（完成図が上の図）
vgg_model = Model(input=vgg16.input, output=top_model(vgg16.output))

# VGG16の図の青色の部分は重みを固定（frozen）
for layer in vgg_model.layers[:15]:
    layer.trainable = False
    
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

vgg_model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 112, 112, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 112, 112, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0         
__________

/home/ryoji/Documents/python/rec_act/scrape/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  # This is added back by InteractiveShellApp.init_path()


In [12]:
vgg_model.fit_generator(datagen.flow(X_train, y_train_, batch_size=batch_size),
                    steps_per_epoch=int(len(X_train) /batch_size),
                    validation_data=datagen.flow(X_val, y_val_, batch_size=batch_size),
                    epochs=epochs)

Epoch 1/5
1872/1872 [==============================] - 208s 111ms/step - loss: 3.6102 - acc: 0.7747 - val_loss: 3.6170 - val_acc: 0.7744
Epoch 2/5
  31/1872 [..............................] - ETA: 2:55 - loss: 3.6521 - acc: 0.7722

KeyboardInterrupt: 

In [16]:
vgg_model.fit(X_train_, t_train_,  batch_size=16, nb_epoch=10, verbose=1, validation_data=(X_val, t_val))

/home/ryoji/Documents/python/rec_act/scrape/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 29960 samples, validate on 7490 samples
Epoch 1/10
29960/29960 [==============================] - 218s 7ms/step - loss: 3.6124 - acc: 0.7746 - val_loss: 3.6170 - val_acc: 0.7744
Epoch 2/10
  784/29960 [..............................] - ETA: 3:00 - loss: 3.8235 - acc: 0.7615

KeyboardInterrupt: 

In [13]:
vgg_model.predict(X_train[-1:])

array([[1., 0.]], dtype=float32)